# Concrete composition analysis using deep learning with Keras

## Aim: 
This project aims to apply a linear regression using a neural network with Keras to analyse data on the composition of concrete


In [2]:
## import the libraries
from numpy import loadtxt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from math import sqrt
import time

2024-07-30 11:51:20.159286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 11:51:20.159384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 11:51:20.285560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# load the dataset
# dataset = loadtxt('/kaggle/input/concrete-data/concrete_data.csv', delimiter=',',skiprows=1)
dataset = pd.read_csv('/kaggle/input/concrete-data/concrete_data.csv')
dataset.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# Data preparation
We need to specify the target for the training, ie the strength of the concrete. 
The variables that affect the strenght are the remaining columns.

In [4]:
# split the data into target and predictors 
dataset_columns = dataset.columns
# all columns except Strength
predictors = dataset[dataset_columns[dataset_columns != 'Strength']] 
target = dataset['Strength'] # Strength column

# Part A: Build a baseline model
Neural network structure: 1 hidden layer with 10 nodes and ReLu activation function

Adam optimiser and mean square error as loss function

In [7]:
# define the keras model for one hidden layer
model = Sequential()
model.add(Dense(10, input_shape=(8,), activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target,test_size=0.3, random_state=42)

In [6]:
# Train the model on the training dataset we created before

model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=10
)

Epoch 1/50
42/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11677.9980 

I0000 00:00:1722329875.910785     208 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 8812.1436 - val_loss: 927.4703
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 913.0039 - val_loss: 624.3834
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 666.7895 - val_loss: 428.2399
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 457.8812 - val_loss: 316.7010
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 348.3760 - val_loss: 243.9630
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 255.7957 - val_loss: 200.0468
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212.5138 - val_loss: 167.6977
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 197.7705 - val_loss: 150.4951
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 152.7487 - val_loss: 145.0193
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 139.5972 - val_loss: 134.2231
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 136.2219 - val_loss: 132.9039
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 

## Evaluate the model on test data

In [7]:
y_pred = model.predict(X_test)
#get the root mean squared error 
RMSE = round(sqrt(MSE(y_test, y_pred)),3)
print("Root mean square between predicted and test data", RMSE)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Root mean square between predicted and test data 10.276


## Repeat the training and evaluation process 50x

In [29]:
iterations=50
RMSE=[] # An array to store the mean squared root values
start = time.time()
for i in range(1,iterations+1):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target,test_size=0.3, random_state=42)
    # We use the verbose=0 option this time to go silent and not get hundreds of line of output
    model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=10,
    verbose=0
    )
    y_pred = model.predict(X_test, verbose=0)
    RMSE.append(round(sqrt(MSE(y_test, y_pred)),3))
    # We don't want too many lines so we only print every 5 iterations
    if i == 1 or i%5 == 0:
        print("Iteration #", i)
        print("Mean squared error: ", RMSE[i-1])
        
end = time.time()
timer = round((end-start)/60,3)
print(f'The model took {timer} minutes to train')

Iteration # 1
Mean squared error:  6.34
Iteration # 5
Mean squared error:  6.232
Iteration # 10
Mean squared error:  6.158
Iteration # 15
Mean squared error:  6.182
Iteration # 20
Mean squared error:  6.171
Iteration # 25
Mean squared error:  6.183
Iteration # 30
Mean squared error:  6.531
Iteration # 35
Mean squared error:  6.737
Iteration # 40
Mean squared error:  6.26
Iteration # 45
Mean squared error:  6.29
Iteration # 50
Mean squared error:  6.194
The model took 6.569 minutes to train


## Compute the mean and std of the mean squared error

In [26]:
RMSE_mean=round(np.mean(RMSE),3)
RMSE_std=round(np.std(RMSE),3)

print(f" After {iterations} iterations, the mean squared error mean is {RMSE_mean} and the std is {RMSE_std} ")

 After 50 iterations, the mean squared error mean is 6.353 and the std is 0.188 


# Part B: analysis using normalised data

We first need to normalise the dataset using a simple formula : norm = (value - mean)/std

In [5]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [9]:
iterations=50
RMSE=[] # An array to store the mean squared root values
start = time.time()
for i in range(1,iterations+1):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target,test_size=0.3, random_state=42)
    # We use the verbose=0 option this time to go silent and not get hundreds of line of output
    model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=10,
    verbose=0
    )
    y_pred = model.predict(X_test, verbose=0)
    RMSE.append(round(sqrt(MSE(y_test, y_pred)),3))
    # We don't want too many lines so we only print every 5 iterations
    if i == 1 or i%5 == 0:
        print("Iteration #", i)
        print("Mean squared error: ", RMSE[i-1])
        
end = time.time()
timer = round((end-start)/60,3)
print(f'The model took {timer} minutes to train')

I0000 00:00:1722340383.108798     126 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Iteration # 1
Mean squared error:  13.495
Iteration # 5
Mean squared error:  9.391
Iteration # 10
Mean squared error:  7.004
Iteration # 15
Mean squared error:  7.061
Iteration # 20
Mean squared error:  7.196
Iteration # 25
Mean squared error:  6.981
Iteration # 30
Mean squared error:  7.02
Iteration # 35
Mean squared error:  6.988
Iteration # 40
Mean squared error:  7.009
Iteration # 45
Mean squared error:  7.826
Iteration # 50
Mean squared error:  7.134
The model took 6.701 minutes to train


Here's the results of the training on normalised data:

In [10]:
RMSE_mean_PartB=round(np.mean(RMSE),3)
RMSE_std_PartB=round(np.std(RMSE),3)

print(f" After {iterations} iterations, the mean squared error mean is {RMSE_mean_PartB} and the std is {RMSE_std_PartB} ")

 After 50 iterations, the mean squared error mean is 7.539 and the std is 1.17 


Now we want to compare the values of the mean squared error mean and std on raw (Part A) and normalised data

Unnormalised data (Part A):
mean = 6.353
std = 0.188

Normalised data:
mean = 7.539 
std = 1.17

The mean is a bit higher and the std much higher.

# Part C: use normalised data ewith 100 epochs for the training

In [11]:
iterations=50
RMSE=[] # An array to store the mean squared root values
start = time.time()
for i in range(1,iterations+1):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target,test_size=0.3, random_state=42)
    # We use the verbose=0 option this time to go silent and not get hundreds of line of output
    model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=10,
    verbose=0
    )
    y_pred = model.predict(X_test, verbose=0)
    RMSE.append(round(sqrt(MSE(y_test, y_pred)),3))
    # We don't want too many lines so we only print every 5 iterations
    if i == 1 or i%5 == 0:
        print("Iteration #", i)
        print("Mean squared error: ", RMSE[i-1])
        
end = time.time()
timer = round((end-start)/60,3)
print(f'The model took {timer} minutes to train')

Iteration # 1
Mean squared error:  7.495
Iteration # 5
Mean squared error:  7.083
Iteration # 10
Mean squared error:  7.255
Iteration # 15
Mean squared error:  7.092
Iteration # 20
Mean squared error:  7.134
Iteration # 25
Mean squared error:  7.108
Iteration # 30
Mean squared error:  7.295
Iteration # 35
Mean squared error:  7.063
Iteration # 40
Mean squared error:  7.097
Iteration # 45
Mean squared error:  7.067
Iteration # 50
Mean squared error:  7.269
The model took 13.265 minutes to train


Here are the results:

In [12]:
RMSE_mean_PartC=round(np.mean(RMSE),3)
RMSE_std_PartC=round(np.std(RMSE),3)

print(f" After {iterations} iterations, the mean squared error mean is {RMSE_mean_PartC} and the std is {RMSE_std_PartC} ")

 After 50 iterations, the mean squared error mean is 7.229 and the std is 0.234 


Now the std is smaller, so using more epochs although it slows down the training 2x, improves significantly the std.

# Part D: add more layers to the neural network

In [14]:
# define the keras model for one hidden layer
model = Sequential()
model.add(Dense(10, input_shape=(8,), activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
iterations=50
RMSE=[] # An array to store the mean squared root values
start = time.time()
for i in range(1,iterations+1):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target,test_size=0.3, random_state=42)
    # We use the verbose=0 option this time to go silent and not get hundreds of line of output
    model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=10,
    verbose=0
    )
    y_pred = model.predict(X_test, verbose=0)
    RMSE.append(round(sqrt(MSE(y_test, y_pred)),3))
    # We don't want too many lines so we only print every 5 iterations
    if i == 1 or i%5 == 0:
        print("Iteration #", i)
        print("Mean squared error: ", RMSE[i-1])
        
end = time.time()
timer = round((end-start)/60,3)
print(f'The model took {timer} minutes to train')

Iteration # 1
Mean squared error:  7.3
Iteration # 5
Mean squared error:  6.898
Iteration # 10
Mean squared error:  6.364
Iteration # 15
Mean squared error:  6.362
Iteration # 20
Mean squared error:  6.25
Iteration # 25
Mean squared error:  6.213
Iteration # 30
Mean squared error:  6.362
Iteration # 35
Mean squared error:  6.639
Iteration # 40
Mean squared error:  6.479
Iteration # 45
Mean squared error:  6.289
Iteration # 50
Mean squared error:  6.324
The model took 7.252 minutes to train


In [16]:
RMSE_mean_PartD=round(np.mean(RMSE),3)
RMSE_std_PartD=round(np.std(RMSE),3)

print(f" After {iterations} iterations, the mean squared error mean is {RMSE_mean_PartD} and the std is {RMSE_std_PartD} ")

 After 50 iterations, the mean squared error mean is 6.489 and the std is 0.288 
